# Comparing Ohsome to OSMnx

In [38]:
import sys
import yaml
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
import seaborn as sns
from ohsome import OhsomeClient
from shapely.geometry import LineString, MultiLineString

sns.set_theme(style="whitegrid")
sns.set_context("paper")

crs=25832
datetime = '2024-03-15T00:00:00Z'

data_dir = Path('./data')
out_dir = Path('./outdir')

## Area of Interest

In [7]:
aoi_gdf = gpd.read_file(data_dir / 'MA-HD-RNK.geojson')
aoi_bpoly = aoi_gdf.to_json()

## Ohsome

In [8]:
client = OhsomeClient()

responses = []
try:
    response = client.elements.geometry.post(
        bpolys = aoi_bpoly, 
        filter = 'highway=* and type:way and geometry:line',
        time = datetime,
        properties = 'tags'
    )
    road_network_gdf = response.as_dataframe()
except Exception as err:
    print(f'Could not send request to ohsome API: {err}')
    sys.exit()

Feature count

In [35]:
len(road_network_gdf)

141586

Column count

In [36]:
len(road_network_gdf.columns)

950

Total road network length

In [39]:
road_network_gdf.to_crs(epsg=crs, inplace=True)

In [40]:
road_network_gdf['road_length'] = round(road_network_gdf['geometry'].length, 2)
f'{round(road_network_gdf["road_length"].sum() / 1000, 2)} km'

'17836.11 km'

Counts of geometry types

In [41]:
road_network_gdf['geometry'].apply(lambda geom: type(geom).__name__).value_counts()

/home/grhalbgott/miniforge3/envs/lulc-ma/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


LineString         141486
MultiLineString        96
Point                   4
Name: count, dtype: int64

## OSMnx

In [27]:
graph = ox.graph_from_polygon(aoi_gdf.unary_union, network_type='all', simplify=False, retain_all=True)
graph_gdf = ox.graph_to_gdfs(graph, nodes=False, edges=True)

Feature count

In [61]:
len(graph_gdf)

1372391

Column count

In [62]:
len(graph_gdf.columns)

18

Total road network length

In [44]:
graph_gdf.to_crs(epsg=crs, inplace=True)

In [45]:
graph_gdf['road_length'] = round(graph_gdf['geometry'].length, 2)
f'{round(graph_gdf["road_length"].sum() / 1000, 2)} km'

'34388.17 km'

Counts of geometry types

In [46]:
graph_gdf['geometry'].apply(lambda geom: type(geom).__name__).value_counts()

/home/grhalbgott/miniforge3/envs/lulc-ma/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


LineString    1372391
Name: count, dtype: int64

### Simplified

In [31]:
simple_graph = ox.simplify_graph(graph)
simple_graph_gdf = ox.graph_to_gdfs(simple_graph, nodes=False, edges=True)

Feature count

In [47]:
len(simple_graph_gdf)

427572

Column count

In [48]:
len(simple_graph_gdf.columns)

18

Total road network length

In [49]:
simple_graph_gdf.to_crs(epsg=crs, inplace=True)

In [50]:
simple_graph_gdf['road_length'] = round(simple_graph_gdf['geometry'].length, 2)
f'{round(simple_graph_gdf["road_length"].sum() / 1000, 2)} km'

'34376.21 km'

Counts of geometry types

In [51]:
simple_graph_gdf['geometry'].apply(lambda geom: type(geom).__name__).value_counts()

/home/grhalbgott/miniforge3/envs/lulc-ma/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


LineString    427572
Name: count, dtype: int64